In [37]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
import numpy as np


In [38]:
import os
print(os.getcwd())


C:\Users\st114\OneDrive\Documents\Fall BU 2025\CS 777 Big Data Analytics\Term Project\wedding-song-recommendations\modeling


In [39]:
# creating spark app
spark = SparkSession.builder.appName("Creating_Clustering_Model").getOrCreate()

#reading in the CSV file 
df = spark.read.csv('numpy_array_for_modeling.csv', header=True, inferSchema=True)

df.printSchema()

root
 |-- rank: double (nullable = true)
 |-- trend: double (nullable = true)
 |-- streams: double (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- af_danceability: double (nullable = true)
 |-- af_energy: double (nullable = true)
 |-- af_key: double (nullable = true)
 |-- af_loudness: double (nullable = true)
 |-- af_mode: double (nullable = true)
 |-- af_speechiness: double (nullable = true)
 |-- af_acousticness: double (nullable = true)
 |-- af_instrumentalness: double (nullable = true)
 |-- af_liveness: double (nullable = true)
 |-- af_valence: double (nullable = true)
 |-- af_tempo: double (nullable = true)
 |-- af_time_signature: double (nullable = true)
 |-- language_id: double (nullable = true)
 |-- date: double (nullable = true)



In [48]:
input_columns =  ["rank","trend","streams","popularity","duration_ms","af_danceability","af_energy","af_key","af_loudness","af_mode","af_speechiness","af_acousticness","af_instrumentalness","af_liveness","af_valence","af_tempo","af_time_signature","language_id","date"]
assembler = VectorAssembler(inputCols = input_columns, outputCol = 'features')
vector_df = assembler.transform(df)

collected_vectors = vector_df.select('features').collect()
#printing one vector
print("Collected Vectors")
print(collected_vectors[:1])

Collected Vectors
[Row(features=DenseVector([1.0, 1.0, 253019.0, 78.0, 195840.0, 0.852, 0.773, 8.0, -2.921, 0.0, 0.0776, 0.187, 0.0, 0.159, 0.907, 102.034, 4.0, 1.0, 2017.0]))]


In [49]:
numpy_feature_matrix = np.asarray([row[0].toArray() for row in collected_vectors])

print('Numpy Feature Matrix from Vector Column:')
print(numpy_feature_matrix)

Numpy Feature Matrix from Vector Column:
[[1.00000e+00 1.00000e+00 2.53019e+05 ... 4.00000e+00 1.00000e+00
  2.01700e+03]
 [2.00000e+00 2.00000e+00 2.23988e+05 ... 4.00000e+00 1.00000e+00
  2.01600e+03]
 [3.00000e+00 0.00000e+00 2.10943e+05 ... 4.00000e+00 1.00000e+00
  2.01600e+03]
 ...
 [1.96000e+02 1.00000e+00 1.43970e+04 ... 4.00000e+00 1.00000e+00
  2.01600e+03]
 [1.97000e+02 0.00000e+00 1.43040e+04 ... 4.00000e+00 0.00000e+00
  2.01500e+03]
 [1.99000e+02 1.00000e+00 1.42320e+04 ... 4.00000e+00 1.00000e+00
  2.01500e+03]]


In [50]:
vector_df.printSchema()

root
 |-- rank: double (nullable = true)
 |-- trend: double (nullable = true)
 |-- streams: double (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- af_danceability: double (nullable = true)
 |-- af_energy: double (nullable = true)
 |-- af_key: double (nullable = true)
 |-- af_loudness: double (nullable = true)
 |-- af_mode: double (nullable = true)
 |-- af_speechiness: double (nullable = true)
 |-- af_acousticness: double (nullable = true)
 |-- af_instrumentalness: double (nullable = true)
 |-- af_liveness: double (nullable = true)
 |-- af_valence: double (nullable = true)
 |-- af_tempo: double (nullable = true)
 |-- af_time_signature: double (nullable = true)
 |-- language_id: double (nullable = true)
 |-- date: double (nullable = true)
 |-- features: vector (nullable = true)



In [51]:
# train the model 
kmeans_model = KMeans(k=2, seed = 42)
model = kmeans.fit(vector_df)

# make predictions
prediction = model.transform(vector_df)
# predictions.show()

In [52]:
prediction.printSchema()

root
 |-- rank: double (nullable = true)
 |-- trend: double (nullable = true)
 |-- streams: double (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- af_danceability: double (nullable = true)
 |-- af_energy: double (nullable = true)
 |-- af_key: double (nullable = true)
 |-- af_loudness: double (nullable = true)
 |-- af_mode: double (nullable = true)
 |-- af_speechiness: double (nullable = true)
 |-- af_acousticness: double (nullable = true)
 |-- af_instrumentalness: double (nullable = true)
 |-- af_liveness: double (nullable = true)
 |-- af_valence: double (nullable = true)
 |-- af_tempo: double (nullable = true)
 |-- af_time_signature: double (nullable = true)
 |-- language_id: double (nullable = true)
 |-- date: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: integer (nullable = false)



In [ ]:
#let's evaluate the clusting by computing within the Set sum of squared errors
wsse = model.summary.trainingCost
print(f"Within Set Sum of Squared Errors = {wsse}")

# show cluster centers
centers= model.clusterCenters()
print('Cluster Centers')
for center in centers:
    print(center)


In [55]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

sil_score = []
evaluator = ClusteringEvaluator(predictionCol = 'prediction',
                               featuresCol = 'features',
                               metricName = 'silhouette',
                               #distanceMeasure = 'squaredEuclidean' 
                               distanceMeasure = 'cosine') #cosine gives us better results
for i in range(2,10):
    kmeans = KMeans(featuresCol = 'features', k = i)
    model = kmeans.fit(vector_df)
    predictions = model.transform(vector_df)
    score = evaluator.evaluate(prediction)
    sil_score.append(score)
    print("Silhouette Score for K = ", i , 'is', score)


Silhouette Score for K =  2 is 0.8385888534419206
Silhouette Score for K =  3 is 0.8385888534419206
Silhouette Score for K =  4 is 0.8385888534419206
Silhouette Score for K =  5 is 0.8385888534419206
Silhouette Score for K =  6 is 0.8385888534419206
Silhouette Score for K =  7 is 0.8385888534419206
Silhouette Score for K =  8 is 0.8385888534419206
Silhouette Score for K =  9 is 0.8385888534419206


Scores without scaling:


Silhouette Score for K =  2 is 0.8385888534419206

Silhouette Score for K =  3 is 0.8385888534419206

Silhouette Score for K =  4 is 0.8385888534419206

Silhouette Score for K =  5 is 0.8385888534419206

Silhouette Score for K =  6 is 0.8385888534419206

Silhouette Score for K =  7 is 0.8385888534419206

Silhouette Score for K =  8 is 0.8385888534419206

Silhouette Score for K =  9 is 0.8385888534419206
